# Task 1: A* Algorithm for Pathfinding and Water Jug Problem

In [ ]:
import heapq

# Grid (0 = free space, 1 = obstacle)
grid = [
    [0, 0, 0, 0, 0],
    [1, 1, 1, 1, 0],
    [0, 0, 0, 1, 0],
    [0, 1, 0, 0, 0],
    [0, 1, 1, 1, 0]
]

# Heuristic function: Manhattan Distance
def manhattan_distance(x1, y1, x2, y2):
    return abs(x1 - x2) + abs(y1 - y2)

# A* Algorithm for Pathfinding
def a_star_search(start, goal, grid):
    rows, cols = len(grid), len(grid[0])
    open_list = []
    heapq.heappush(open_list, (0, start))  # (f, (x, y))
    came_from = {}
    g_score = {start: 0}

    while open_list:
        _, current = heapq.heappop(open_list)
        if current == goal:
            return reconstruct_path(came_from, current)

        x, y = current
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            neighbor = (x + dx, y + dy)
            if 0 <= neighbor[0] < rows and 0 <= neighbor[1] < cols and grid[neighbor[0]][neighbor[1]] == 0:
                tentative_g_score = g_score[current] + 1
                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score = tentative_g_score + manhattan_distance(neighbor[0], neighbor[1], goal[0], goal[1])
                    heapq.heappush(open_list, (f_score, neighbor))

    return None  # If no path is found

# Reconstruct the path
def reconstruct_path(came_from, current):
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    return path[::-1]

# Main execution
start = (0, 0)
goal = (4, 4)
path = a_star_search(start, goal, grid)

# Print the grid with the path marked as 'P'
if path:
    for (x, y) in path:
        grid[x][y] = 'P'
    for row in grid:
        print(' '.join(map(str, row)))
else:
    print("No path found")


P P P P P
1 1 1 1 P
0 0 0 1 P
0 1 0 0 P
0 1 1 1 P


# Task2: A* Algorithm for Water Jug Problem

In [ ]:
import heapq

# Heuristic function: absolute difference from target (2 liters in either jug)
def heuristic(jug1, jug2):
    return abs(jug1 - 2)

# A* Algorithm for Water Jug Problem
def a_star_water_jug(start, target):
    open_list = []
    heapq.heappush(open_list, (0, start))  # (f, (jug1, jug2))
    came_from = {}
    g_score = {start: 0}
    visited = set()

    while open_list:
        _, current = heapq.heappop(open_list)
        jug1, jug2 = current

        if jug1 == target or jug2 == target:
            return reconstruct_jug_path(came_from, current)

        if current in visited:
            continue
        visited.add(current)

        # Possible actions (fill, empty, pour)
        next_states = [
            (4, jug2),  # Fill jug1
            (jug1, 3),  # Fill jug2
            (0, jug2),  # Empty jug1
            (jug1, 0),  # Empty jug2
            (max(0, jug1 - (3 - jug2)), min(3, jug2 + jug1)),  # Pour jug1 into jug2
            (min(4, jug1 + jug2), max(0, jug2 - (4 - jug1)))   # Pour jug2 into jug1
        ]

        for next_state in next_states:
            if next_state not in visited:
                tentative_g_score = g_score[current] + 1
                if next_state not in g_score or tentative_g_score < g_score[next_state]:
                    came_from[next_state] = current
                    g_score[next_state] = tentative_g_score
                    f_score = tentative_g_score + heuristic(next_state[0], next_state[1])
                    heapq.heappush(open_list, (f_score, next_state))

    return None  # No solution found

# Reconstruct the path
def reconstruct_jug_path(came_from, current):
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    return path[::-1]

# Main execution
start = (0, 0)
target = 2
path = a_star_water_jug(start, target)

# Print the solution path
if path:
    print("Solution path:")
    for state in path:
        print(f"Jug1: {state[0]} liters, Jug2: {state[1]} liters")
else:
    print("No solution found")


Solution path:
Jug1: 0 liters, Jug2: 0 liters
Jug1: 0 liters, Jug2: 3 liters
Jug1: 3 liters, Jug2: 0 liters
Jug1: 3 liters, Jug2: 3 liters
Jug1: 4 liters, Jug2: 2 liters


# Task 3: Mini-Max Algorithm for Tic-Tac-Toe

In [ ]:
import math

# Function to check for a winner
def evaluate(board):
    # Rows, Columns, and Diagonals
    win_conditions = [(board[0], board[1], board[2]),
                      (board[3], board[4], board[5]),
                      (board[6], board[7], board[8]),
                      (board[0], board[3], board[6]),
                      (board[1], board[4], board[7]),
                      (board[2], board[5], board[8]),
                      (board[0], board[4], board[8]),
                      (board[2], board[4], board[6])]

    for line in win_conditions:
        if line == ('X', 'X', 'X'):
            return 1  # 'X' wins
        if line == ('O', 'O', 'O'):
            return -1  # 'O' wins
    return 0  # No winner yet

# Check if there are any moves left
def is_moves_left(board):
    return any(s == ' ' for s in board)

# Mini-Max function to determine the best score for both players
def minimax(board, depth, is_max):
    score = evaluate(board)

    # Terminal conditions: either someone won, or no moves left
    if score == 1:
        return score
    if score == -1:
        return score
    if not is_moves_left(board):
        return 0  # Tie

    # Maximizing player's move (X)
    if is_max:
        best = -math.inf
        for i in range(9):
            if board[i] == ' ':
                board[i] = 'X'
                best = max(best, minimax(board, depth + 1, False))
                board[i] = ' '
        return best
    # Minimizing player's move (O)
    else:
        best = math.inf
        for i in range(9):
            if board[i] == ' ':
                board[i] = 'O'
                best = min(best, minimax(board, depth + 1, True))
                board[i] = ' '
        return best

# Find the best move for the AI
def find_best_move(board, is_max):
    best_val = -math.inf if is_max else math.inf
    best_move = -1

    for i in range(9):
        if board[i] == ' ':
            board[i] = 'X' if is_max else 'O'
            move_val = minimax(board, 0, not is_max)
            board[i] = ' '

            if is_max:
                if move_val > best_val:
                    best_move = i
                    best_val = move_val
            else:
                if move_val < best_val:
                    best_move = i
                    best_val = move_val

    return best_move

# Display the current Tic-Tac-Toe board
def print_board(board):
    for i in range(0, 9, 3):
        print(f'{board[i]} | {board[i + 1]} | {board[i + 2]}')
        if i < 6:
            print('---------')

# Get the user's move
def get_user_move(board):
    move = -1
    while move < 0 or move > 8 or board[move] != ' ':
        try:
            move = int(input("Enter your move (0-8): "))
            if board[move] != ' ':
                print("That space is already taken. Try again.")
        except ValueError:
            print("Invalid input. Please enter a number between 0 and 8.")
    return move

# Main game loop for Tic-Tac-Toe
def play_game():
    board = [' ' for _ in range(9)]  # Initialize the board (empty spaces)

    print("Welcome to Tic-Tac-Toe!")
    user_symbol = input("Choose your symbol (X or O): ").upper()
    ai_symbol = 'O' if user_symbol == 'X' else 'X'
    user_turn = user_symbol == 'X'

    print(f"You are '{user_symbol}', AI is '{ai_symbol}'.")
    print_board(board)

    while is_moves_left(board):
        if user_turn:
            # User's move
            print("\nYour turn:")
            user_move = get_user_move(board)
            board[user_move] = user_symbol
            print_board(board)
            if evaluate(board) == 1 if user_symbol == 'X' else -1:
                print(f"{user_symbol} wins!")
                return
            user_turn = False
        else:
            # AI's move
            print("\nAI's turn:")
            ai_move = find_best_move(board, ai_symbol == 'X')
            board[ai_move] = ai_symbol
            print_board(board)
            if evaluate(board) == 1 if ai_symbol == 'X' else -1:
                print(f"{ai_symbol} wins!")
                return
            user_turn = True

        if not is_moves_left(board):
            break

    print("It's a draw!")

# Run the game
play_game()


Welcome to Tic-Tac-Toe!
Choose your symbol (X or O): X
You are 'X', AI is 'O'.
  |   |  
---------
  |   |  
---------
  |   |  

Your turn:
Enter your move (0-8): 0
X |   |  
---------
  |   |  
---------
  |   |  

AI's turn:
X |   |  
---------
  | O |  
---------
  |   |  
O wins!
